# BMG Labtech CLARIOstar Plus

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Optical Systems:</b> Dual LVF Monochromator, Physical Filters, UV/Vis Spectrometer (220–1000 nm combined range)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Rapid full-plate autofocus, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

**CLARIOstar Plus vs CLARIOstar:** The CLARIOstar Plus (post-2019) replaced the original CLARIOstar. Both share the dual LVF Monochromator + filter + spectrometer architecture. The Plus adds rapid full-plate autofocus, newer PMT options (e.g. far-red), and Voyager control software. EDR (Enhanced Dynamic Range) was introduced after 2024 and is **not** present on all CLARIOstar Plus units. This backend targets the CLARIOstar Plus but may also work with the original CLARIOstar (untested).

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar Plus, create a `PlateReader` frontend
instance that uses a `CLARIOstarPlusBackend` as its backend.

Currently supported: `setup()`, `open()`, `close()`, `stop()`, status polling,
device identification (EEPROM, firmware, usage counters), and temperature control.
Measurement commands (absorbance, fluorescence, luminescence) will be added in later phases.

In [1]:
from pylabrobot import verbose

verbose(True)

In [2]:
from pylabrobot.plate_reading import PlateReader
from pylabrobot.plate_reading.bmg_labtech import CLARIOstarPlusBackend

clariostar_plus_backend = CLARIOstarPlusBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_plus_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0,
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_plus_backend = CLARIOstarPlusBackend(device_id="FT1234AB")
```

In [3]:
await pr.setup()

2026-02-24 10:37:28,443 - pylabrobot.io.ftdi - INFO - Successfully opened FTDI device: 430-2621
2026-02-24 10:37:28,701 - pylabrobot - INFO - read 24 bytes: 0200180c011500040000030000000000000000c00001030d
2026-02-24 10:37:28,702 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 10:37:28,739 - pylabrobot - INFO - read 24 bytes: 0200180c010500040000000000000000000000c00000f00d
2026-02-24 10:37:28,740 - pylabrobot - INFO - status: {'standby': False, 'busy': False, 'running': False, 'unread_data': False, 'initialized': False, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None}
2026-02-24 10:37:28,813 - pylabrobot - INFO - read 271 bytes: 02010f0c070500040000000100000a0101010100000100ee0200000f00e2030000000000000304000001000001020000000000000000000032000000000000000000000000000000000000000074006f0000000000000065000000dc050000000000000000f4010803a70408076009da08ac0d00000000000000000

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration,
initialize command). The backend polls status until the `initialized` flag is set.
```

---
### Defining & Assigning a Plate

Every measurement requires a plate to be assigned to the plate reader (to tell the firmware what the positions for reading are).

In [4]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [5]:
# Open the drawer (plate out)
await pr.open()

2026-02-24 10:37:28,925 - pylabrobot - INFO - read 24 bytes: 0200180c012500040000030000000000000000c00001130d
2026-02-24 10:37:28,933 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 10:37:28,974 - pylabrobot - INFO - read 24 bytes: 0200180c012500240000030000000000000000c00001330d
2026-02-24 10:37:28,975 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None}
2026-02-24 10:37:29,112 - pylabrobot - INFO - read 24 bytes: 0200180c012500240000030000000000000000c00001330d
2026-02-24 10:37:29,113 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None}
2026-02-24 10:37:29,251 - pylabrobot - INFO - rea

In [6]:
# Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
await pr.close()

2026-02-24 10:37:33,177 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000c00001300d
2026-02-24 10:37:33,178 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 10:37:33,216 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000c00001300d
2026-02-24 10:37:33,217 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': True, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None}
2026-02-24 10:37:33,355 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000c00001300d
2026-02-24 10:37:33,356 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': True, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None}
2026-02-24 10:37:33,493 - pylabrobot - INFO - read 

---
## Device Identification

EEPROM configuration, firmware info, usage counters, and status queries.

In [7]:
config = clariostar_plus_backend.configuration
for key, value in config.items():
    print(f"  {key:25s} {value}")

print()
modes = await clariostar_plus_backend.request_available_detection_modes()
print(f"  Available detection modes: {', '.join(modes) if modes else 'none'}")

2026-02-24 10:37:42,031 - pylabrobot - INFO - read 271 bytes: 02010f0c070500240000000100000a0101010100000100ee0200000f00e2030000000000000304000001000001020000000000000000000032000000000000000000000000000000000000000074006f0000000000000065000000dc050000000000000000f4010803a70408076009da08ac0d0000000000000000000000000000000000000000000000000100000001010000000000000001010000000000000012029806ae013d0a4605ee01fbff700c00000000a40058ff8e03f20460ff5511fe0b55118f1a170298065aff970668042603bc14b804080791009001463228460a0046071e0000000000000000002103d40628002c01900146001e00001411001209ac0d6009000000000020260d
2026-02-24 10:37:42,032 - pylabrobot - INFO - EEPROM: 263 bytes, head=070500240000000100000a0101010100


  serial_number             430-2621
  firmware_version          1.35
  firmware_build_timestamp  Nov 20 2020 11:51:21
  model_name                Unknown BMG reader (type 0x0004)
  machine_type_code         4
  max_temperature           45
  has_absorbance            True
  has_fluorescence          True
  has_luminescence          True
  has_alpha_technology      True
  excitation_monochromator_max_nm 750
  emission_monochromator_max_nm 994
  excitation_filter_slots   4
  dichroic_filter_slots     3
  emission_filter_slots     4

  Available detection modes: absorbance, absorbance_spectrum, fluorescence, luminescence, alpha_technology


In [8]:
counters = await clariostar_plus_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

2026-02-24 10:37:42,084 - pylabrobot - INFO - read 50 bytes: 0200320c210500240000001e023d0000078c000004e7000003d9000275cf000012fa0000000a0000000a0000000a0005b10d


  flashes                      1,966,653
  testruns                         1,932
  wells                          125,500
  well_movements                  98,500
  active_time_s                  161,231
  shake_time_s                     4,858
  pump1_usage                         10
  pump2_usage                         10
  alpha_time                          10


In [9]:
status = await clariostar_plus_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

2026-02-24 10:37:42,134 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


  standby              False
  busy                 False
  running              False
  unread_data          False
  initialized          True
  drawer_open          False
  plate_detected       False
  temperature_bottom   None
  temperature_top      None


In [10]:
if await clariostar_plus_backend.request_plate_detected():
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_plus_backend.is_ready():
    print("Machine is ready")
else:
    print("Machine is busy")

2026-02-24 10:37:42,217 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,257 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


No plate detected
Machine is ready


---
## Temperature

Temperature monitoring and incubation control.

In [11]:
temp = await clariostar_plus_backend.measure_temperature()
print(f"Bottom plate: {temp:.1f} °C")

temp_top = await clariostar_plus_backend.measure_temperature(sensor="top")
print(f"Top plate:    {temp_top:.1f} °C")

temp_mean = await clariostar_plus_backend.measure_temperature(sensor="mean")
print(f"Mean:         {temp_mean:.1f} °C")

2026-02-24 10:37:42,313 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,351 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,401 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,449 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,490 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,531 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,569 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,607 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:42,645 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

Bottom plate: 22.8 °C


2026-02-24 10:37:42,990 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,029 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,071 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,109 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,147 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,185 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,223 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,261 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000e400e8c00002dc0d
2026-02-24 10:37:43,301 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

Top plate:    23.2 °C


2026-02-24 10:37:43,493 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,531 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,569 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,609 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,647 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,685 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,723 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:37:43,761 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000e400e8c00002dc0d


Mean:         23.0 °C


In [13]:
import asyncio

await clariostar_plus_backend.start_temperature_control(37.0)

for _ in range(35):
    temp = await clariostar_plus_backend.measure_temperature()
    print(f"Bottom plate: {temp:.1f} °C")
    await asyncio.sleep(2)

await clariostar_plus_backend.stop_temperature_control()

2026-02-24 10:38:03,608 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:03,646 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:03,685 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:03,723 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:03,762 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000e400e8c00002dc0d


Bottom plate: 22.8 °C


2026-02-24 10:38:05,809 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:05,847 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:05,885 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:05,923 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:05,961 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:06,001 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:06,040 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:06,078 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:06,116 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

Bottom plate: 22.8 °C


2026-02-24 10:38:08,310 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,348 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,386 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,424 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,462 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,500 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,538 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,576 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:38:08,614 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

...

2026-02-24 10:39:28,817 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:28,855 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:28,893 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:28,931 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:28,969 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:29,007 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:29,045 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:29,083 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:29,121 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

Bottom plate: 22.8 °C


2026-02-24 10:39:31,314 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,352 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,390 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,427 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,465 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,503 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,542 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,580 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 10:39:31,618 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24

Bottom plate: 22.8 °C


2026-02-24 10:39:33,810 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


---
## Planned Features

The sections below cover measurement modes that are not yet implemented.
Code cells are commented out and will be uncommented as each feature lands.

### Absorbance

Single-wavelength, multi-wavelength, transmittance, partial well selection.

In [ ]:
# results = await pr.read_absorbance(
#     wavelength=600,
# )
#
# print(f"OD at 600nm, well A1: {results[0]['data'][0][0]}")

In [ ]:
# results = await pr.read_absorbance(
#     wavelength=450,
#     report="transmittance",
# )
#
# # Transmittance is in percent (0-100)
# print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

In [ ]:
# results = await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600, 750],
# )
#
# for r in results:
#     print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]:.4f}")

In [ ]:
# column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     wells=column_1_wells,
# )

### Fluorescence

Basic, custom gain/bandwidth, bottom-optic.

In [ ]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
# )
#
# print(f"GFP fluorescence, well A1: {results[0]['data'][0][0]}")

In [ ]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
#     gain=1500,
#     ex_bandwidth=10,
#     em_bandwidth=20,
#     flashes=50,
# )

In [ ]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=4.5,
#     bottom_optic=True,
# )

### Luminescence

Basic luminescence, partial well selection.

In [ ]:
# results = await pr.read_luminescence(
#     focal_height=13.0,
# )
#
# print(f"Temperature: {results[0]['temperature']:.1f} °C")
# print(f"Well A1 RLU: {results[0]['data'][0][0]}")

In [ ]:
# selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     wells=selected_wells,
# )

### Advanced Features

Scan modes, shaking, non-blocking reads, combined features.

In [ ]:
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import StartCorner
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     start_corner=StartCorner.BOTTOM_RIGHT,
#     unidirectional=True,
#     vertical=True,
# )

In [ ]:
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import ShakerType
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=5,
# )

In [ ]:
# import asyncio
#
# await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600],
#     wait=False,
# )
#
# status = await clariostar_plus_backend.request_machine_status()
# while not status["unread_data"]:
#     print("Measuring...")
#     await asyncio.sleep(1)
#     status = await clariostar_plus_backend.request_machine_status()
#
# results = await clariostar_plus_backend.collect_absorbance_measurement(plate)

In [ ]:
# row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]
#
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=6.0,
#     wells=row_A,
#     gain=2000,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=3,
#     start_corner=StartCorner.BOTTOM_RIGHT,
# )

---
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts**
and the backend converts them to transmittance and then to OD.
See `DESIGN.md` in `pylabrobot/plate_reading/bmg_labtech/` for the full protocol reference.

---
## Closing Connection

In [ ]:
pr.unassign_child_resource(plate)

In [ ]:
await pr.stop()

This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.